# YouTube Knowledge Base

Para esta caso, utilizaremos o Whisper como modelos de conversão de voz em texto. O texto extraído dos vídeos serão  armazenados, junto aos seus embeddings.

Para instalar o whisper, utilizaremos o código do github (e não o pacote whisper disponível no pip).

In [ ]:
# Install whisper.ai
!pip install git+https://github.com/openai/whisper.git --quiet

In [ ]:
# Install ffmeg
#Mac
!brew upgrade && brew install ffmpeg
# Unix
#!sudo apt update && sudo apt install ffmpeg

### GPU vs CPU

Se estiver executando em um ambiente com GPU, a conversão de audio em texto será mais rápida

In [1]:
import torch
torch.cuda.is_available()
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

'cpu'

## Instalando dependencias adicionais

In [ ]:
!pip install pytube moviepy mock pydub python-dotenv google-cloud-aiplatform tqdm  --quiet

In [ ]:
!pip install langchain --upgrade --quiet

In [ ]:
!pip install astrapy --upgrade --quiet

In [2]:
from dotenv import load_dotenv, find_dotenv
import os
load_dotenv(find_dotenv(), override=True)
table_name="demo_vertex"

# Youtube Extraction

Este caso começa com a extração de conteúdo do youtube.

Para isso, algumas funções são necessárias. Elas vão desde o download do vídeo, extração do aúdio em blocos, extração de texto dos áudios.

A sequência toda está na função process_youtube

In [3]:
import os
import pytube
from tqdm import tqdm
from moviepy.editor import *
from pydub import AudioSegment

In [4]:
def split_mp3(input_file, output_directory, segment_duration_sec, overlap_sec = 1):
    segments = []

    # Load the MP3 file
    audio = AudioSegment.from_mp3(input_file)
    segment_duration_ms = segment_duration_sec * 1000
    overlap_ms = overlap_sec * 1000
    # Calculate the number of segments
    num_segments = len(audio) // ( segment_duration_ms - overlap_ms)

    # Create the output directory if it doesn't exist
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Split the MP3 file into segments
    for i in range(num_segments):
        start_time = i * ( segment_duration_ms - overlap_ms)
        if start_time < 0:
          start_time = 0

        end_time = (i + 1) * segment_duration_ms
        segment = audio[start_time:end_time]

        # Define the output filename
        output_file = os.path.join(output_directory, f"segment_{str(i + 1).zfill(5)}.mp3")

        # Export the segment as an MP3 file
        segment.export(output_file, format="mp3")
        segments.append({
            "start_time": start_time / 1000,
            "end_time": end_time / 1000,
            "audio_file": output_file
        })


        print(f"Segment {i + 1} saved as {output_file}")

    return segments


In [39]:
# Solve pytube issue: https://github.com/pytube/pytube/issues/1498
import re
import mock

from pytube.cipher import get_throttling_function_code

def patched_throttling_plan(js: str):
    """Patch throttling plan, from https://github.com/pytube/pytube/issues/1498"""
    raw_code = get_throttling_function_code(js)

    transform_start = r"try{"
    plan_regex = re.compile(transform_start)
    match = plan_regex.search(raw_code)

    #transform_plan_raw = find_object_from_startpoint(raw_code, match.span()[1] - 1)
    transform_plan_raw = js

    # Steps are either c[x](c[y]) or c[x](c[y],c[z])
    step_start = r"c\[(\d+)\]\(c\[(\d+)\](,c(\[(\d+)\]))?\)"
    step_regex = re.compile(step_start)
    matches = step_regex.findall(transform_plan_raw)
    transform_steps = []
    for match in matches:
        if match[4] != '':
            transform_steps.append((match[0],match[1],match[4]))
        else:
            transform_steps.append((match[0],match[1]))

    return transform_steps

def download_youtube_audio(video_url, output_path):
  try:
    with mock.patch('pytube.cipher.get_throttling_plan', patched_throttling_plan):
      import pytube
      # Create a YouTube object
      yt = pytube.YouTube(video_url)

      # Get the highest resolution stream
      video_stream = yt.streams.filter(only_audio=True).first()

      # Download the audio stream
      video_stream.download(output_path=output_path)

      # Convert the downloaded file to MP3
      mp4_file_path = os.path.join(output_path, video_stream.default_filename)
      mp3_file_path = os.path.splitext(mp4_file_path)[0] + ".mp3"

      video_clip = AudioFileClip(mp4_file_path)
      video_clip.write_audiofile(mp3_file_path)

      # Delete the original MP4 file
      os.remove(mp4_file_path)

      print(f"Audio downloaded and saved as {mp3_file_path}")
      return mp3_file_path
  except Exception as e:
    print(f"An error occurred: {str(e)}")

In [5]:
import whisper
import numpy as np
import pandas as pd

import torch
torch.cuda.is_available()
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

model = whisper.load_model("medium", device=DEVICE)
print(
    f"Model is {'multilingual' if model.is_multilingual else 'English-only'} "
    f"and has {sum(np.prod(p.shape) for p in model.parameters()):,} parameters."
)

Model is multilingual and has 762,321,920 parameters.


In [6]:
def decode_audio(segments):
  segments_out = []
  import whisper
  import numpy as np
  print("Loading model")
  model = whisper.load_model("medium", device=DEVICE)
  res = []
  for segment in tqdm(segments, total=len(segments), desc="Extracting texts from segments"):
    # print(segment)
    audio = whisper.load_audio(segment['audio_file'])
    audio = whisper.pad_or_trim(audio)
    mel = whisper.log_mel_spectrogram(audio).to(model.device)
    options = whisper.DecodingOptions(language="pt", fp16 = False)
    result = whisper.decode(model, mel, options)
    res.append({
        **segment,
        'text': result.text
    })
  return res

In [7]:
def process_youtube(url):
    print("Downloading")
    audio_file = download_youtube_audio(url,".")
    print("Extracting Audio")
    segments = split_mp3(audio_file,"audio",29)
    print("Converting to text")
    return segments
    # decoded = decode_audio(segments)
    # return decoded

In [8]:
url = "https://www.youtube.com/watch?v=rteFpZi__Bo"

In [40]:
segments = process_youtube(url)

Downloading
MoviePy - Writing audio in ./PAULO MUZY REVELOU OS 6 MAIORES MITOS DA CREATINA !!!.mp3


MoviePy - Done.
Audio downloaded and saved as ./PAULO MUZY REVELOU OS 6 MAIORES MITOS DA CREATINA !!!.mp3
Extracting Audio
Segment 1 saved as audio/segment_00001.mp3
Segment 2 saved as audio/segment_00002.mp3
Segment 3 saved as audio/segment_00003.mp3
Segment 4 saved as audio/segment_00004.mp3
Segment 5 saved as audio/segment_00005.mp3
Segment 6 saved as audio/segment_00006.mp3
Segment 7 saved as audio/segment_00007.mp3
Segment 8 saved as audio/segment_00008.mp3
Segment 9 saved as audio/segment_00009.mp3
Segment 10 saved as audio/segment_00010.mp3
Segment 11 saved as audio/segment_00011.mp3
Segment 12 saved as audio/segment_00012.mp3
Segment 13 saved as audio/segment_00013.mp3
Segment 14 saved as audio/segment_00014.mp3
Segment 15 saved as audio/segment_00015.mp3
Segment 16 saved as audio/segment_00016.mp3
Segment 17 saved as audio/segment_00017.mp3
Segment 18 saved as audio/segment_00018.mp3
Segment 19 saved as audio/segment_00019.mp3
Segment 20 saved as audio/segment_00020.mp3
Segmen

In [41]:
decoded = decode_audio(segments)

Loading model


Extracting texts from segments: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 22/22 [05:06<00:00, 13.94s/it]


In [42]:
# Visualizando o conteúdo extraído.
df = pd.DataFrame(decoded)
df.head()

,start_time,end_time,audio_file,text
0,0.0,29.0,audio/segment_00001.mp3,"Fala galera da Max Paul Muzio aqui, e olha só,..."
1,28.0,58.0,audio/segment_00002.mp3,que essas pessoas depois elas tinham a sua cap...
2,56.0,87.0,audio/segment_00003.mp3,da creatina em doses até maiores do que aquele...
3,84.0,116.0,audio/segment_00004.mp3,"a caixinha, mesmo que você não faça exercício,..."
4,112.0,145.0,audio/segment_00005.mp3,massa muscular até uma certa idade e que o env...


# Vextex AI Setup

In [ ]:
# GCP Initialization 
!gcloud init 
!gcloud services enable aiplatform.googleapis.com     
!gcloud auth application-default set-quota-project os.environ['GCP_PROJECT_ID']

In [9]:
# Setup do projeto no Vertex AI
import vertexai
vertexai.init(project = os.environ['GCP_PROJECT_ID'],
              location = os.environ['GCP_REGION_ID'])

### Gerando embeddings

In [10]:
from vertexai.preview.language_models import TextEmbeddingModel
model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")

In [11]:
emb = model.get_embeddings(["Astra DB é um banco de dados de vetores baseado no Apache Cassandra"])[0].values

In [12]:
len(emb)

768

In [13]:
emb[:10]

[-0.08548425137996674,
 -0.020405586808919907,
 -0.012789517641067505,
 0.012251014821231365,
 0.06244432553648949,
 -0.05537176504731178,
 -0.003829806111752987,
 -0.02250230312347412,
 -0.033818479627370834,
 -0.025387773290276527]

In [14]:
def get_embedding(text):
    return model.get_embeddings([text])[0].values

# DataStax Astra DB Vector

Para utilizar o Astra DB Vector, instalaremos a lib ASTRAPY

In [ ]:
pip install astrapy --quiet --upgrade

In [15]:
from astrapy.db import AstraDB

In [16]:
from getpass import getpass

In [17]:
ASTRA_DB_API_ENDPOINT = input("ASTRA_DB_API_ENDPOINT = ")
ASTRA_DB_APPLICATION_TOKEN = getpass("ASTRA_DB_APPLICATION_TOKEN = ")

ASTRA_DB_API_ENDPOINT = https://b0748576-a92d-4682-86b0-13a0a04fb4dd-us-east1.apps.astra.datastax.com
ASTRA_DB_APPLICATION_TOKEN = ········


In [18]:
# Criando o client
astra_db = AstraDB(
    api_endpoint=ASTRA_DB_API_ENDPOINT,
    token=ASTRA_DB_APPLICATION_TOKEN,
)

In [ ]:
# collection_name="assistant_youtube"
# astra_db.delete_collection(collection_name)

In [19]:
# Criando a collection
collection_name="assistant_youtube"
collection = astra_db.create_collection(collection_name, 
                                        dimension=768, # Determinada pelo tamanho do embedding gerado pelo modelo
                                        metric="DOT_PRODUCT" # Métrica de similaridade => EUCLIDEAN, DOT_PRODUCT, COSINE
                                       )

In [20]:
import urllib.parse as urlparse
def get_video_id(url):
    url_data = urlparse.urlparse(url)
    query = urlparse.parse_qs(url_data.query)
    return query["v"][0]

In [51]:
# Removendo todos os dados (se necessário)
# collection_name="assistant_youtube"
# astra_db.truncate_collection(collection_name)

Astra DB Collection[name="assistant_youtube", endpoint="https://b0748576-a92d-4682-86b0-13a0a04fb4dd-us-east1.apps.astra.datastax.com"]

In [52]:
%time
# Insere os chunks do video no Astra DB Vector 

for index, row in tqdm(df.iterrows(), total=df.shape[0], desc=f'Loading DF'):
    try:            
        v_doc = {
            "_id": f"{get_video_id(url)}#{str(index).rjust(3,'0')}",
            "content": row["text"],
            "metadata" : {
                "influencer": "Paulo Muzy",
                "source": f"{url}&t={row['start_time']:.0f}",
            },
            "$vector": get_embedding(row["text"]),
        }

        response = collection.insert_one(v_doc)
        #print(response)        
    except Exception as e:
        print(f"Error at IX {index}", e)

print("Finished")

CPU times: user 3 µs, sys: 2 µs, total: 5 µs
Wall time: 10 µs


Loading DF: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 22/22 [00:13<00:00,  1.69it/s]

Finished


## Busca semântica

In [45]:
query = "A creatina faz mal?"

In [46]:
# Executando uma busca com vector_find

documents = collection.vector_find(
    get_embedding(query), 
    limit=3,
    # filter={"influencer": "Paulo Muzy"}, ## Filtrando documentos para melhor performance
    # fields=["text", "$vector"], ## Escolhendo os campos a serem retornados  
    # include_similarity=False
)

df_vector_search = pd.DataFrame(documents)
df_vector_search.head()

,_id,content,metadata,source,$vector,$similarity
0,rteFpZi__Bo#015,O título do artigo é esse. A subunitação de cr...,{'influencer': 'Paulo Muzy'},https://www.youtube.com/watch?v=rteFpZi__Bo&t=420,"[0.0024520978331565857, -0.0589134655892849, -...",0.877457
1,rteFpZi__Bo#001,que essas pessoas depois elas tinham a sua cap...,{'influencer': 'Paulo Muzy'},https://www.youtube.com/watch?v=rteFpZi__Bo&t=28,"[-0.004914171528071165, -0.01098625548183918, ...",0.876461
2,rteFpZi__Bo#014,"Porque parece assim, ah, mas você está falando...",{'influencer': 'Paulo Muzy'},https://www.youtube.com/watch?v=rteFpZi__Bo&t=392,"[-0.008156763389706612, -0.022999338805675507,...",0.871836


# Obtendo respostas

Utilizando os modelos disponíveis no Vertex AI para gerar embeddings e textos.

In [24]:

from vertexai.language_models import TextGenerationModel, \
                                     TextEmbeddingModel, \
                                     ChatModel, \
                                     InputOutputTextPair, \
                                     CodeGenerationModel, \
                                     CodeChatModel, \
                                    ChatMessage

In [25]:
generation_model = TextGenerationModel.from_pretrained("text-bison@001")

In [26]:
# Testing the LLM
query = "O que é creatina?"
response = generation_model.predict(prompt=query)
print(response.text)

A creatina é um composto orgânico que é produzido naturalmente no corpo humano e que também pode ser encontrado em alimentos como carne, peixe e aves. A creatina é usada pelo corpo para produzir energia, especialmente durante atividades físicas de alta intensidade.

A creatina é um suplemento popular entre atletas e pessoas que se exercitam regularmente, pois pode ajudar a melhorar o desempenho físico e a recuperação muscular. Estudos mostram que a suplementação com creatina pode aumentar a força muscular, a potência e a resistência, além de reduzir o tempo de recuperação após o exercício.

A creatina é geralmente considerada segura, mas algumas pessoas podem experimentar efeitos


In [27]:
def get_content(query):
    content = ""
    results = collection.vector_find(
        get_embedding(query), 
        limit=3)
    for r in results:
        # print(f"Conteúdo: {r['body_blob']}")
        # print(f"Similaridade: {r['distance']}")
        # print(f"Metadata: {r['metadata']}\n")
        content += f"""Conteúdo: {r['text']}
Source: {r['source']}
        
"""
    return content
    

In [28]:
query = "Quais os benefícios da creatina?"

content = get_content(query)

prompt = f"""Responda a pergunta do usuário de maneira simples com base somente no conteúdo abaixo. 

ORIGEM
{content}

PERGUNTA:
{query}"""

print("Prompt:")
print(prompt)

response = generation_model.predict(prompt=prompt)

print(response.text)

Prompt:
Responda a pergunta do usuário de maneira simples com base somente no conteúdo abaixo. 

ORIGEM
Conteúdo: que essas pessoas depois elas tinham a sua capacidade cognitiva e psicomotora medida. E o que foi encontrado? Foi encontrado que depois de 24 horas de privação do sono, a suplementação de creatina tinha um efeito positivo, principalmente no estado de humor das pessoas e nas tarefas que elas tinham que realizar. Isso é uma coisa que eu falo para vocês há algum tempo. Para alguns pacientes que eu tenho, eu uso creatina ou recomendo o uso da creatina em
Source: https://www.youtube.com/watch?v=rteFpZi__Bo&t=28
        
Conteúdo: da creatina para conseguir melhorar a qualidade de vida e sintomas maiores quando você tem a doença na forma mais grave. Falando em gravidade, estudo do grupo do Lancha Junior, um rapaz chamado Carlos Ferraz, estudo exploratório sobre potenciais efeitos anti-câncer de creatina, ou seja, aqui a gente tem literatura
Source: https://www.youtube.com/watch?v

# Chat model

Modelos de chat buscam lidar com histórico de mensagens

In [29]:
chat_model = ChatModel.from_pretrained("chat-bison@001")

content_chat = get_content(query)

chat = chat_model.start_chat(
    context=f"""Você responde de maneira divertida questões sobre suplementos alimentares, e se limita somente a este CONTEXT. 
    CONTEXT:
    {content_chat}
    """,
    temperature=0.5,
    max_output_tokens=300,
    top_p=0.8,
    top_k=4,
    message_history = [
            ChatMessage(content="Gostaria de tirar dúvidas sobre suplementos", author="user"),
            ChatMessage(content="Ótimo, eu sou um agente que responde sobre suplementos alimentares e somente sobre isso", author="bot"),
            ]
)

In [30]:
query = "A creatina faz mal?"
res = chat.send_message(query)
res

Não, a creatina não faz mal, muito pelo contrário, ela é um suplemento muito seguro e que pode trazer muitos benefícios para a saúde.

# Com LangChain

Langchain é o framework mais utilizado atualmente para o desenvolvimento de aplicações IA Generativa.

In [61]:
# Importamos o Astra DB como Vector Store
from langchain.vectorstores import AstraDB

In [32]:
from langchain.llms import VertexAI
from langchain.chat_models import ChatVertexAI
from langchain.embeddings import VertexAIEmbeddings
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

In [33]:
llm = VertexAI()
print(llm("O que são suplementos alimentares?"))

 Os suplementos alimentares são produtos destinados a complementar a dieta alimentar e que contêm nutrientes como vitaminas, minerais, fibras alimentares, aminoácidos, ácidos graxos essenciais e outras substâncias com efeito fisiológico. Eles podem ser encontrados em diversas formas, como comprimidos, cápsulas, pós, líquidos e barras.

Os suplementos alimentares não são medicamentos e não devem ser usados para tratar ou prevenir doenças. No entanto, eles podem ser úteis para pessoas que não conseguem obter os nutrientes necessários apenas com a dieta alimentar. Por exemplo, pessoas que seguem dietas restritivas, como vegetarianos e veganos, podem precisar de suplementos


In [34]:
embedding_generator = VertexAIEmbeddings()

In [54]:
AstraVectorStore = AstraDB(
    embedding=embedding_generator,
    collection_name=collection_name,
    api_endpoint=ASTRA_DB_API_ENDPOINT,
    token=ASTRA_DB_APPLICATION_TOKEN,
    metric="DOT_PRODUCT"
)

index = VectorStoreIndexWrapper(
    vectorstore=AstraVectorStore
)

In [59]:
from langchain.globals import set_debug
set_debug(False)
print("\nResposta com o query_with_sources:")
print(query)
print(index.query_with_sources(question=query, verbose=True))


Resposta com o query_with_sources:
A creatina faz mal?


> Entering new RetrievalQAWithSourcesChain chain...

> Finished chain.
{'question': 'A creatina faz mal?', 'answer': ' Não há evidências de que a creatina seja prejudicial.\n', 'sources': 'https://www.youtube.com/watch?v=rteFpZi__Bo&t=420, https://www.youtube.com/watch?v=rteFpZi__Bo&t=28, https://www.youtube.com/watch?v=rteFpZi__Bo&t=392, https://www.youtube.com/watch?v=rteFpZi__Bo&t=280'}


In [56]:
set_debug(False)
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain import PromptTemplate

prompt_template = """
Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). 
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
ALWAYS return a "SOURCES" part in your answer. Answer in Portuguese.


QUESTION: {question}
=========
{summaries}
=========
FINAL ANSWER:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["summaries", "question"]
)

In [58]:
set_debug(False)
retrieverSim = AstraVectorStore.as_retriever(search_kwargs={"k": 3})
# Create a "RetrievalQA" chain
chainSim = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retrieverSim,
    chain_type_kwargs={
        'prompt': PROMPT,
        'document_variable_name': 'summaries'
    }
)
# Run it and print results
responseSim = chainSim.run(query)
print(responseSim)

 Não, a creatina não faz mal. Pelo contrário, ela pode até mesmo trazer benefícios para a saúde, como melhora do humor e da função cognitiva. 
Alguns estudos mostram que a creatina pode causar problemas renais em pacientes que fazem treinamento resistido e consomem dietas hiperproteicas. No entanto, esses estudos são limitados e não há evidências suficientes para afirmar que a creatina é prejudicial aos rins. 
De acordo com o Dr. Lair Ribeiro, a creatina é uma molécula sensacional, que tem uma série de aplicações. Ele recomenda o uso da creatina para
